In [25]:
from pyspark.sql.functions import desc
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import pyspark

In [2]:
spark = (SparkSession.builder.config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.2-s_2.12').appName("lab6").getOrCreate())

:: loading settings :: url = jar:file:/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mina/.ivy2/cache
The jars for the packages stored in: /home/mina/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa5096f7-195f-41a5-a57a-ab28a75adf8e;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 434ms :: artifacts dl 10ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	------------------------------------

1- create df

In [7]:
stations = spark.read.option("header","true").csv("station_data.csv")
stations.show(5)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|San Jose|    8/7/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 5 rows



In [8]:
trips = spark.read.option("header","true").csv("trip_data.csv") 
trips.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

2- rename name column to id

In [10]:
vertices = stations.withColumnRenamed("name", "id").distinct()
vertices.show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|
+----------+--------------------+---------+-----------+---------+-------------+------------+
only showing top 5 rows



3- rename start/end station to src and dst

In [12]:
edges = trips.withColumnRenamed("Start Station", "src").withColumnRenamed("End Station", "dst") 
edges.show(5)

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

4- graphFrame

In [18]:
graph = GraphFrame(vertices, edges)
graph.edges.show(5)

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [19]:
graph.vertices.show(5)

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|        51|Embarcadero at Fo...|37.791464|-122.391034|       19|San Francisco|   8/20/2013|
|        58|San Francisco Cit...| 37.77865|-122.418235|       19|San Francisco|   8/21/2013|
|        60|Embarcadero at Sa...| 37.80477|-122.403234|       15|San Francisco|   8/21/2013|
|        65|     Townsend at 7th|37.771058|-122.402717|       15|San Francisco|   8/22/2013|
|        63|       Howard at 2nd|37.786978|-122.398108|       19|San Francisco|   8/22/2013|
+----------+--------------------+---------+-----------+---------+-------------+------------+
only showing top 5 rows



5- the nbr of trips made between each src and dst (DESC)

In [20]:
graph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show(10) 

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|San Francisco Cal...|    3|
|San Francisco Cal...|  Powell Street BART|    2|
|     2nd at Townsend|   Market at Sansome|    2|
|     Spear at Folsom|     2nd at Townsend|    2|
|   Market at Sansome|Broadway St at Ba...|    2|
|    Davis at Jackson|Embarcadero at Sa...|    2|
|San Francisco Cal...|   2nd at South Park|    2|
|   Steuart at Market|San Francisco Cal...|    2|
|       Market at 4th|San Francisco Cal...|    2|
+--------------------+--------------------+-----+
only showing top 10 rows



6- the nbr of trips that start/end at "Townsend at 7th" station (DESC)

In [21]:
graph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'").groupBy("src", "dst").count().orderBy(desc("count")).show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|     Townsend at 7th|    1|
|     Townsend at 7th|     Spear at Folsom|    1|
|     Townsend at 7th|Harry Bridges Pla...|    1|
|     Spear at Folsom|     Townsend at 7th|    1|
|       Howard at 2nd|     Townsend at 7th|    1|
+--------------------+--------------------+-----+



7- the vertices that have never been a dst for a trip starting from "Spear at Folsom"

In [24]:
graph.edges.select("dst").distinct().subtract(graph.edges.where("src = 'Spear at Folsom'").select("dst").distinct()).show(10, False)

+---------------------------------------------+
|dst                                          |
+---------------------------------------------+
|Powell at Post (Union Square)                |
|Market at Sansome                            |
|Spear at Folsom                              |
|San Salvador at 1st                          |
|Rengstorff Avenue / California Street        |
|Civic Center BART (7th at Market)            |
|Mountain View City Hall                      |
|Market at 4th                                |
|Temporary Transbay Terminal (Howard at Beale)|
|Steuart at Market                            |
+---------------------------------------------+
only showing top 10 rows



8- the station with max nbr of incoming trips

In [28]:
graph.edges.groupBy("dst").agg(count("*").alias("incoming_trips")).orderBy(col("incoming_trips").desc()).limit(1).show(1, False) 

+---------------------------------------+--------------+
|dst                                    |incoming_trips|
+---------------------------------------+--------------+
|San Francisco Caltrain 2 (330 Townsend)|9             |
+---------------------------------------+--------------+



9- trip with the longest duration

In [29]:
graph.edges.orderBy(desc("Duration")).limit(1).show()

+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|           src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913448|     932|8/31/2015 21:57|Post at Kearny|            47|8/31/2015 22:12|South Van Ness at...|          66|   472|     Subscriber|   94702|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+



10- subgraph that contains only trips that start/end at "Townsend at 7th"

In [30]:
subgraph = GraphFrame(graph.vertices, graph.edges.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'"))

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


11- all paths that form triangle pattern between three station

In [31]:
graph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)").show(10, False)

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|a                                                                                               |ab                                                                                                                                       |b                                                                         

In [32]:
subgraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)").show(10, False)

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+---+---+---+---+---+
|a  |ab |b  |bc |c  |ca |
+---+---+---+---+---+---+
+---+---+---+---+---+---+



12- all paths that pass through three vertices and start from "Townsend at 7th"

In [37]:
graph.find("(a)-[]->(b); (b)-[]->(c)").filter("a.id = 'Townsend at 7th'").show(10, False)

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|a                                                                          |b                                                                                               |c                                                                                                  |
+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|{65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013}|{50, Harry Bridges Plaza (Ferry Building), 37.795392, -122.394203, 23, San Francisco, 8/20/2013}|{

In [40]:
subgraph.find("(a)-[]->(b); (b)-[]->(c)").filter("a.id = 'Townsend at 7th'").show(10, False)

/home/mina/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---------------------------------------------------------------------------+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|a                                                                          |b                                                                          |c                                                                          |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------+---------------------------------------------------------------------------+
|{65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013}|{49, Spear at Folsom, 37.790302, -122.390637, 19, San Francisco, 8/20/2013}|{65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013}|
+---------------------------------------------------------------------------+---